In [12]:
#portfolio optimization espanol
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
#pip install pandas_datareader –––-> Para instalar el paquete pandas_datereader.
from pandas_datareader import data
from pulp import *
#!pip install PuLP
import py
    

from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.cla import CLA
#from pypfopt.plotting import Plotting
from matplotlib.ticker import FuncFormatter
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [13]:
activos = ["TSLA", "RIVN", "SPY", "PYPL", "GOOGL" ]

fechaInicio = "2016-04-20"
hoy = datetime.today().strftime('%Y-%m-%d')

df_precios = pd.DataFrame()

In [14]:
def datosYahoo(dataframe,nombresActivos,inicio,fin):
    for i in nombresActivos:
        dataframe[i] = data.DataReader(i,data_source='yahoo',start=inicio , end=fin)["Adj Close"]
    return dataframe
df = datosYahoo(df_precios,activos,fechaInicio,hoy)
df

TypeError: string indices must be integers

In [ ]:
plt.figure(figsize=(12.2,4.5)) 
for i in df.columns.values:
    plt.plot( df[i],  label=i)
plt.title('Precio de las Acciones')
plt.xlabel('Fecha',fontsize=18)
plt.ylabel('Precio en USD',fontsize=18)
plt.legend(df.columns.values, loc='upper left')
plt.savefig('plotprecios.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df = np.log(df).diff()
df = df.dropna()
df

In [ ]:
plt.figure(figsize=(12.2,4.5)) 
for i in df.columns.values:
    plt.hist( df[i],  label=i, bins = 200)
plt.title('Histograma de los retornos')
plt.xlabel('Fecha',fontsize=18)
plt.ylabel('Precio en USD',fontsize=18)
plt.legend(df.columns.values)
plt.savefig('plotretornosnormales.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Separamos el Benchmark del resto de los activos creando una nueva base de datos.
df_activos =  df.loc[:, df.columns != 'SPY']
df_activos

In [ ]:
df_benchmark1 =  df.loc[:, df.columns == 'SPY']

In [ ]:
retornos1 = expected_returns.capm_return(df_activos, market_prices = df_benchmark1, returns_data= True, risk_free_rate=0.07/100, frequency=252)
retornos1

In [ ]:
def pesosPortafolio(dataframe):
    array = []
    for i in dataframe.columns:
        array.append(1/len(dataframe.columns))
    arrayFinal = np.array(array)
    return arrayFinal
pesos = pesosPortafolio(df_activos)
pesos

In [ ]:
df_cov = df_activos.cov()*252
df_cov

In [ ]:
#Varianza del Portafolio
varianza_portafolio = pesos.T @ df_cov @pesos
"La varianza del portafolio es:" + " " + str(round(varianza_portafolio*100,1))+"%"

In [ ]:
volatilidad_portafolio = np.sqrt(varianza_portafolio)
"La volatilidad del portafolio es:" + " " + str(round(volatilidad_portafolio*100,1))+"%"
#Resultado:
'La volatilidad del portafolio es: 23.5%'

In [ ]:
retorno_portafolio = np.sum(pesos*retornos1)
'El retorno anual del portafolio es:' + ' ' + str(round(retorno_portafolio*100,3)) + '%'
# Resultado: 
'El retorno anual del portafolio es: 18.723%'

In [ ]:
## Optimización para portafolio de mínima varianza.
ef = EfficientFrontier(retornos1, df_cov, weight_bounds=(-1,1))
weights = ef.min_volatility() 
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) 
ef.portfolio_performance(verbose=True)

In [ ]:
## Realizamos el mismo procedimiento que la vez anterior, pero esta vez solo aceptamos pesos positivos (weight_bounds=(0,1)).
ef = EfficientFrontier(retornos1, df_cov, weight_bounds=(0,1))
weights = ef.min_volatility() 
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) 
ef.portfolio_performance(verbose=True)

In [ ]:
##Optimización para el Sharpe Ratio: con venta corta.
#Para maximizar el Sharpe ratio debemos cambiar el min.volatility por ef.max_sharpe().

ef = EfficientFrontier(retornos1, df_cov, weight_bounds=(-1,1))
weights = ef.max_sharpe() 
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) 
ef.portfolio_performance(verbose=True)

In [ ]:
#Optimización para el Sharpe Ratio: sin venta corta.

ef = EfficientFrontier(retornos1, df_cov, weight_bounds=(-1,1))
weights = ef.max_sharpe() 
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) 
ef.portfolio_performance(verbose=True)

In [4]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [5]:
latest_prices = get_latest_prices(df_activos)
latest_prices

NameError: name 'df_activos' is not defined

In [6]:
pesos = cleaned_weights 
print(pesos)

NameError: name 'cleaned_weights' is not defined

In [7]:
da = DiscreteAllocation(pesos, latest_prices, total_portfolio_value=300)
print(da)

NameError: name 'pesos' is not defined

In [8]:

allocation, leftover = da.greedy_portfolio(reinvest=True, verbose=False)
print(allocation)

NameError: name 'da' is not defined

In [9]:
print("Cantidad de acciones a comprar:", allocation)
print("Dinero sobrante: ${:.2f}".format(leftover))

NameError: name 'allocation' is not defined

In [10]:
correlation_mat = df.corr()
plt.figure(figsize=(12.2,4.5))
sns.heatmap(correlation_mat, annot = True)
plt.title('Matriz de Correlación')
plt.xlabel('Activos',fontsize=18)
plt.ylabel('Activos',fontsize=18)
plt.show()

NameError: name 'df' is not defined

In [11]:
plt.show()